### Set the stage

In [1]:
import pandas as pd
from IPython.display import display

positions = pd.DataFrame(columns=['ticker', 'shares'], data=[['VTI',87], ['VXUS', 103], ['VNQ',14], ['VOO',6],['VBK', 5]])
closing = pd.DataFrame(columns=['ticker', 'price'], data=[['VTI',149.73], ['VXUS',51.12],['VNQ', 93.43], ['VOO',270.37],['VBK', 181.52]])
target = pd.DataFrame(columns=['ticker', 'target_allocation'], data=[['VTI',47], ['VXUS', 34], ['VNQ',5], ['VOO',10],['VBK', 4]])

In [24]:
from functools import reduce
my_portfolio = reduce(lambda left,right: pd.merge(left,right,on='ticker'), [positions, closing, target])
my_portfolio['market_value'] = my_portfolio['shares']*my_portfolio['price']
total_value = my_portfolio.market_value.sum()
my_portfolio['current_allocation'] = 100 * my_portfolio['market_value'] / total_value
display(my_portfolio)
print("Total taget allocation {}, total value {}".format(my_portfolio.target_allocation.sum(),total_value))

[Optimal lazy rebalancing](http://optimalrebalancing.tk/explanation.html)

### Step 1: Calculate the fractional deviations

Define the fractional deviation `f` of an asset to be `a/t − 1`, where `t` is the asset's target allocation and a is its actual portion of the portfolio. Calculate `f` for each asset. `f` will be negative for underweighted assets and positive for overweighted assets. Note that a denotes the portion relative to the final total portfolio value; this is obtained by adding the contribution amount to the original total portfolio value.

In [25]:
def calc_deviation(portfolio, sorting = True):
    portfolio['deviation']=portfolio.current_allocation/portfolio.target_allocation -1
    if sorting:
        portfolio.sort_values(by=['deviation'], inplace=True)

calc_deviation(my_portfolio)
display(my_portfolio)

### Step 2: Add money to asset(s) with lowest fractional deviation

Add money to the asset(s) with least f until they are tied with the asset(s) with the next-least f. The money added to each asset must be proportional to that asset's target allocation so that the minimum f's increase in synchrony. Repeat this until the contribution amount is exhausted. If the assets are pre-sorted according to f, this process can be implemented such that its running time increases linearly with the number of assets.


In [20]:
print("cheapeast price per share {}".format(my_portfolio.price.min()))
print(my_portfolio.index)
print(my_portfolio.index.to_list())
my_portfolio.loc[my_portfolio.index[0],:]

In [22]:
def rebalance(portfolio, money):
    def one_take(cur_asset, next_asset):
        i = 1
        while portfolio.loc[cur_asset,'deviation'] < portfolio.loc[next_asset,'deviation']:
            if money < portfolio.loc[cur_asset,'price']:
                # money exhausted
                return
            portfolio.loc[cur_asset,'market_value'] += portfolio.loc[cur_asset,'price']
            portfolio.loc[cur_asset,'rebalanced_shares'] += 1
            print('total value {}'.format(portfolio.market_value.sum()))
            portfolio.loc[cur_asset,'current_allocation'] = portfolio.loc[cur_asset,'market_value']/ portfolio.market_value.sum()
            display(portfolio.loc[cur_asset,])
            print("cur d {}, next d {}".format(portfolio.loc[cur_asset,'deviation'], portfolio.loc[next_asset,'deviation']))
            i+=1
            if i > 5:
                return
            calc_deviation(portfolio,False)

    
    cheapest = my_portfolio.price.min()
    portfolio['rebalanced_shares'] = portfolio['shares']
    while money >= cheapest:
        it = iter(my_portfolio.index.to_list())
        cur_asset_index = next(it)
        for next_asset_index in it:
            one_take(cur_asset_index, next_asset_index)
            return
        calc_deviation(portfolio)

In [23]:
rebalance(my_portfolio, 1000)
display(my_portfolio)